<a href="https://colab.research.google.com/github/pikaduck/twt-sentiment-analysis/blob/main/notebooks/twt_sentiment_analysis_setfit.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
! pip install setfit

In [ ]:
from sklearn.model_selection import train_test_split
import pandas as pd
from datasets import Dataset

In [29]:
df_train = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/train.csv')
df_test = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/test.csv')

In [ ]:
df_train.head(3)

,sentiment,idx,timestamp,device,username,tweet,cleaned_tweet,numbers_cleaned_tweet,punc_cleaned_tweet,lemmatized_stop_removed_tweet
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t...","- awww, that is a bummer. you shoulda got davi...","- awww, that is a bummer. you shoulda got davi...",awww that is a bummer you shoulda got david ca...,awww bummer shoulda got david carr third day
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...,is upset that he can not update his facebook b...,is upset that he can not update his facebook b...,is upset that he can not update his facebook b...,upset not update facebook texting might cry re...
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...,i dived many times for the ball. managed to sa...,i dived many times for the ball. managed to sa...,i dived many times for the ball managed to sav...,dived many time ball managed save rest go bound


In [30]:
df_train = pd.concat([
    df_train.loc[df_train['sentiment'] == 0].sample(frac = 1).iloc[:64],
    df_train.loc[df_train['sentiment'] == 4].sample(frac = 1).iloc[:64]
  ])

df_train['sentiment'] = df_train['sentiment'].replace(4, 1)
df_test = df_test.loc[df_test['sentiment'] != 2]
df_test['sentiment'] = df_test['sentiment'].replace(4, 1)

df_train['label_text'] = df_train['sentiment'].apply(lambda x : 'positive' if x==1 else 'negative')
df_test['label_text'] = df_test['sentiment'].apply(lambda x : 'positive' if x==1 else 'negative')

df_train = df_train.rename(columns = {'sentiment' : 'label', 'cleaned_tweet' : 'text'})
df_test = df_test.rename(columns = {'sentiment' : 'label', 'cleaned_tweet' : 'text'})

df_train = df_train[['text', 'label', 'label_text']]
df_test = df_test[['text', 'label', 'label_text']]

<ipython-input-30-546674732f2b>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_test['sentiment'] = df_test['sentiment'].replace(4, 1)
<ipython-input-30-546674732f2b>:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_test['label_text'] = df_test['sentiment'].apply(lambda x : 'positive' if x==1 else 'negative')


In [31]:
train_dataset = Dataset.from_pandas(df_train, split = 'train')
test_dataset = Dataset.from_pandas(df_test, split = 'test')

In [32]:
train_dataset = train_dataset.remove_columns('__index_level_0__')
test_dataset = test_dataset.remove_columns('__index_level_0__')

In [33]:
train_dataset

Dataset({
    features: ['text', 'label', 'label_text'],
    num_rows: 128
})

In [34]:
from sentence_transformers.losses import CosineSimilarityLoss

from setfit import SetFitModel, SetFitTrainer

model = SetFitModel.from_pretrained("sentence-transformers/paraphrase-mpnet-base-v2")

model_head.pkl not found on HuggingFace Hub, initialising classification head with random weights. You should TRAIN this model on a downstream task to use it for predictions and inference.


In [35]:
trainer = SetFitTrainer(
    model = model,
    train_dataset = train_dataset,
    eval_dataset = test_dataset,
    loss_class = CosineSimilarityLoss,
    batch_size = 16,
    num_iterations = 20,
    num_epochs = 3
)

In [36]:
trainer.train()

***** Running training *****
  Num examples = 5120
  Num epochs = 3
  Total optimization steps = 960
  Total train batch size = 16


Epoch:   0%|          | 0/3 [00:00<?, ?it/s]

Iteration:   0%|          | 0/960 [00:00<?, ?it/s]

Iteration:   0%|          | 0/960 [00:00<?, ?it/s]

Iteration:   0%|          | 0/960 [00:00<?, ?it/s]

In [37]:
trainer.evaluate()

***** Running evaluation *****


{'accuracy': 0.8551532033426184}

In [16]:
model.save_pretrained('/content/drive/MyDrive/Colab Notebooks/models/setfit-classifier')

In [17]:
model = SetFitModel.from_pretrained('/content/drive/MyDrive/Colab Notebooks/models/setfit-classifier')

In [40]:
model(['My my, the way Le Bron James played today was surreal']).item()

1

In [42]:
! du -sh /content/drive/MyDrive/Colab\ Notebooks/models/setfit-classifier

419M	/content/drive/MyDrive/Colab Notebooks/models/setfit-classifier
